In [17]:
#importing libraries
import ssl
import requests
from bs4 import BeautifulSoup
import difflib
import csv
import pandas as pd 
#Preparing context for handeling SSL certificate errors
ctx=ssl.create_default_context()
ctx.check_hostname=False
ctx.verify_mode=ssl.CERT_NONE

In [19]:
#Importing datasets from csv file
genre_set= pd.read_csv('movies.csv')
rating_set=pd.read_csv('ratings.csv')

In [25]:
genre_set.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [48]:
#Dividing movie name and year
value=genre_set.iloc[0].title
print(value)
name,year=value.split('(')
year=year[:-1]
print(name,year)
print(len(genre_set))

Toy Story (1995)
Toy Story  1995
9742


In [59]:
name=genre_set['title'].str.split('(')
genre_set['name']=[value[0][:-1] for value in name]
print(genre_set['name'])

0                                Toy Story
1                                  Jumanji
2                         Grumpier Old Men
3                        Waiting to Exhale
4              Father of the Bride Part II
                       ...                
9737    Black Butler: Book of the Atlantic
9738                 No Game No Life: Zero
9739                                 Flint
9740          Bungo Stray Dogs: Dead Apple
9741          Andrew Dice Clay: Dice Rules
Name: name, Length: 9742, dtype: object


In [89]:
value=genre_set.iloc[29]['title'].split('(')
print(value)

['Shanghai Triad ', 'Yao a yao yao dao waipo qiao) ', '1995)']


In [91]:
temp_val=list()
for temp in name:
    #print(temp[1]) #There are not only single string name but also multiple strings
    temp_val.append(temp[-1][:-1])
#print(temp_val)
genre_set['year']=temp_val

In [92]:
genre_set.head()

,movieId,title,genres,name,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II,1995


In [93]:
#Now work with ratings csv file
ratings_set=pd.read_csv('ratings.csv')
ratings_set.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [110]:
#Perform join operation with genre_set for movie name
ratings_genre=genre_set[['movieId','name','year','genres']].merge(ratings_set,on='movieId',how='inner')

In [111]:
ratings_genre

,movieId,name,year,genres,userId,rating,timestamp
0,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483
...,...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic,2017,Action|Animation|Comedy|Fantasy,184,4.0,1537109082
100832,193583,No Game No Life: Zero,2017,Animation|Comedy|Fantasy,184,3.5,1537109545
100833,193585,Flint,2017,Drama,184,3.5,1537109805
100834,193587,Bungo Stray Dogs: Dead Apple,2018,Action|Animation,184,3.5,1537110021


In [ ]:
final_average=